# Model

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.model_selection import train_test_split

<h3 style= "color:blue;"> Importing Data Set </h3>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# load your dataframe
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Research/Method 2 - Titles/preprocessed_title.csv",index_col=[0])

In [6]:
df.head()

,Title,classes
0,aarogya setu mobile application monitor mitiga...,6
1,online teach ict post covid,9
2,best method family plan text mining analysis m...,6
3,mobile industry impact sustainable goal,7
4,recommendation governance guide lawmaker regul...,1


In [7]:
# change both types into string
df['Title'] = df['Title'].astype('str')

In [2]:
!pip install tensorflow tensorflow_datasets transformers

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification

<b> Before moving ahead let's convert the fileclass into integer format, as it will make our work easy!!</b>

In [9]:
# Assuming you have your data in a DataFrame named `df`
train_data = df['Title'].tolist()
train_labels = df['classes'].tolist()

# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(
    train_data, train_labels, test_size=0.3, random_state=42
)

In [10]:
from keras.utils import to_categorical

In [11]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode the training data
train_encodings = tokenizer(train_data, truncation=True, padding=True, max_length=100, return_tensors='tf')

# Tokenize and encode the testing data
test_encodings = tokenizer(test_data, truncation=True, padding=True, max_length=100, return_tensors='tf')

In [12]:
# Convert the labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(train_labels)
test_labels = tf.convert_to_tensor(test_labels)

# Load the pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=12)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Fine-tune the model on your training data
model.fit(train_encodings['input_ids'], train_labels, epochs=10, batch_size=64)

# Make predictions on the test data
predicted_labels = model.predict(test_encodings['input_ids']).logits.argmax(axis=1)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
9/9 [==============================] - 302s 27s/step - loss: 2.4636 - accuracy: 0.1119
Epoch 2/10
9/9 [==============================] - 243s 27s/step - loss: 2.3775 - accuracy: 0.2025
Epoch 3/10
9/9 [==============================] - 244s 27s/step - loss: 2.3326 - accuracy: 0.2131
Epoch 4/10
9/9 [==============================] - 237s 26s/step - loss: 2.3010 - accuracy: 0.2185
Epoch 5/10
9/9 [==============================] - 227s 25s/step - loss: 2.2372 - accuracy: 0.2504
Epoch 6/10
9/9 [==============================] - 225s 25s/step - loss: 2.1528 - accuracy: 0.2558
Epoch 7/10
9/9 [==============================] - 226s 25s/step - loss: 2.0316 - accuracy: 0.3357
Epoch 8/10
9/9 [==============================] - 228s 25s/step - loss: 1.9251 - accuracy: 0.3641
Epoch 9/10
9/9 [==============================] - 226s 25s/step - loss: 1.7950 - accuracy: 0.4600
Epoch 10/10
8/8 [==============================] - 29s 3s/step


NameError: ignored

In [13]:
from sklearn.metrics import classification_report

In [14]:
# Generate classification report
report = classification_report(test_labels, predicted_labels)

print(report)

              precision    recall  f1-score   support

           0       0.78      0.70      0.74        10
           1       0.44      0.67      0.53        12
           2       0.00      0.00      0.00         7
           3       0.18      0.48      0.26        27
           4       0.68      0.77      0.72        62
           5       0.25      0.03      0.06        29
           6       0.20      0.04      0.07        24
           7       0.00      0.00      0.00        13
           8       0.00      0.00      0.00         8
           9       0.00      0.00      0.00        16
          10       0.14      0.32      0.20        25
          11       0.00      0.00      0.00         9

    accuracy                           0.36       242
   macro avg       0.22      0.25      0.21       242
weighted avg       0.31      0.36      0.30       242



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
